# 如何生成 *.rec 文件
以性别、年龄两种属性的 multi-task 为例

1. 首先生成 attr_train.lst 和 attr_valid.lst，每一条记录的格式为
```
# id    gender_label  age     filename
 342         0       32    data/aa.png
```
2. 生成 *.rec 文件
```
python im2rec.py --train-ratio=1 --num-thread=24 --pack-label=1 attr_train.lst  data_root
python im2rec.py --train-ratio=1 --num-thread=24 --pack-label=1 attr_train.lst  data_root
```
其中 --pack-label=1 是必须要设定的，表示有多个标签需要 pack。

# 构建 DataIter 迭代器 

In [2]:
import mxnet as mx

class custom_iter(mx.io.DataIter):
    def __init__(self, data_iter, label_names=None):
        super().__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size
        self.label_names = label_names     # label_names = ['softmax_label', 'focal_label']
    
    @property
    def provide_data(self):
        return self.data_iter.provide_data
    
    @property
    def provide_label(self):
        provide_label = self.data_iter.provide_label[0]
        return [(self.label_names[0], [provide_label[1][0]]),
                (self.label_names[1], [provide_label[1][0]])]
    
    def hard_reset(self):
        self.data_iter.hard_reset()
    
    def reset(self):
        self.data_iter.reset()
    
    def next(self):
        batch = self.data_iter.next()
        label = batch.label[0].asnumpy()
        label1 = mx.nd.array(label[:, 0]).astype('float32')   # label of gender
        label2 = mx.nd.array(label[:, 1]).astype('float32')   # label of age
        return mx.io.DataBatch(data=batch.data, label=[label1, label2], pad=batch.pad, index=batch.index)

# 构建评价指标

In [4]:
import mxnet as mx
import numpy as np

class MultiMetric(mx.metric.EvalMetric):
    def __init__(self, num=2):
        super().__init__('multi-task')
        self.num = num
    
    def get(self):
        """get the current evaluation name and result"""
        if self.num == 2:
            names = ['%s_%s' % (self.name, i) for i in ['gender-acc', 'age-mse']]
        values = [x/y if y != 0 else float('nan') for x, y in zip(self.sum_metric, self.num_inst)]
        return names, values   
    
    def reset(self):
        """override reset behaviors"""
        if getattr(self, 'num', None) is None:
            self.num_inst = 0
            self.sum_metric = 0.0
        else:
            self.num_inst = [0] * self.num
            self.sum_metric = [0.0] * self.num
    
    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        if self.num is None:
            assert len(labels) == self.num
            
        for i in range(len(labels)):
            pred = mx.nd.argmax_channel(preds[i].asnumpy().astype('int32'))
            label = labels[i].asnumpy().astype('int32')
            mx.metric.check_label_shapes(label, pred)
            
            if i is None:
                self.sum_metric += (pred.flat == label.flat).sum()
                self.num_inst += len(pred.flat)
            else:
                if i == 0:
                    # accuracy
                    self.sum_metric[i] += (pred.flat == label.flat).sum()
                    self.num_inst[i] += len(pred.flat)
                else:
                    # mse
                    self.sum_metric[i] += np.sum(np.power(label - pred, 2))
                    self.num_inst[i] += len(pred.flat)                    

# 转换 Training Model 为 Deploy Model

In [ ]:
def get_new_arguments(net, arg_params, aux_params):
    arg_names = net.list_arguments()
    new_args = dict()
    new_auxs = dict()
    for k, v in arg_params.items():
        if k in arg_names:
            new_args[k] = v
    for k, v in aux_params.items():
        new_auxs[k] = v
    return new_args, new_auxs

def convert_model(model_path, deploy_path, optimal_epoch=0):
    model_prefix = model_path + 'mobilenet'
    save_prefix = deploy_path + '/mobilenet'
    save_epoch = optimal_epoch
    sym, arg_params, aux_params = mx.model.load_checkpoints(model_prefix, optimal_epoch)
    all_layers = sym.get_internals()
    net1 = all_layers['fc1_output']    # gender
    net2 = all_layers['fc2_output']    # age
    net = mx.symbol.Group(net1, net2)  # merge gender and age
    new_arg_params, new_aux_params = get_new_arguments(net, arg_params, aux_params)
    save_callback = mx.callback.do_chackpoint(save_prefix)
    save_callback(save_epoch, net, new_arg_params, new_aux_params)
    
    # # Way 2
    # new_arg_params, new_aux_params = get_new_arguments(net, arg_params, aux_params)
    # mx.model.save_checkpoint(save_prefix, save_epoch, net, new_arg_params, new_aux_params)